In [1]:
import os
import torch
import numpy as np
import pandas as pd
from CellClass import CNN
from natsort import natsorted
import matplotlib.pyplot as plt
from CellClass.CNN import dataset
import CellClass.CNN.dataset as D
import CellClass.CNN.training as T
from torch.utils.data import DataLoader

In [2]:
save_dir = "/home/simon_g/src/MICCAI/trained_models"

setups = os.listdir(save_dir)

models = []
for setup in setups:
    files = [os.path.join(save_dir, setup, x) for x in os.listdir(os.path.join(save_dir, setup))]
    if any([x for x in files if ".png" in x]):
        models.extend([x for x in files if ".pt" in x])
    
losses = []
for model in models:
    dict_mod = torch.load(model)
    losses.append({"model": model, "loss": dict_mod["validation_loss"]})

In [3]:
model_df = pd.DataFrame(losses)

In [4]:
sorted = model_df.sort_values("loss")
best = sorted.iloc[0,:]
print(best)
print(best.model)

model    /home/simon_g/src/MICCAI/trained_models/202206...
loss                                              0.000072
Name: 9, dtype: object
/home/simon_g/src/MICCAI/trained_models/20220623_170114/CNN_Model_20220623_212156710.pt


In [5]:
best_setup = best.model.split("/")[-2]
with open(os.path.join(save_dir, best_setup, "training_parameters.txt"), "r") as fin:
    lines = fin.readlines()

    lines = [x.strip().split(":") for x in lines]
    lines = np.array(lines)
    
setup_df = pd.DataFrame(lines.T[1,:], index=lines.T[0,:], columns=[0]).T
setup_df = setup_df.loc[:,~setup_df.columns.duplicated()].copy()

In [6]:
try:
    layers = np.array(setup_df.layers.item().strip().replace("[", "").replace("]", "").split(",")).astype(int)
    in_shape = np.array(setup_df.in_shape.item().strip().replace("[", "").replace("]", "").split(",")).astype(int)

except:
    layers = np.array([3, 16, 64, 128, 256])
    in_shape = np.array([128, 128])

In [16]:
#best = torch.load(best.model, map_location="cpu")
best = torch.load("/home/simon_g/src/MICCAI/trained_models/20220623_172030/CNN_Model_20220624_085936784.pt", map_location="cpu")
model = CNN.ClassificationCNN()
model.load_state_dict(best["model_state_dict"])

<All keys matched successfully>

In [17]:
patches_dir = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/MYCN_SpikeIn/results/single_patches"
dilutions = np.unique([x.split("_")[0]+"_" for x in os.listdir(patches_dir)])
print(dilutions)

['S11_' 'S12_' 'S19_' 'S1b_' 'S20_' 'S29_' 'S2_' 'S30_' 'S31_' 'S32_'
 'S33_' 'S34_' 'S3_' 'S4_' 'S6_' 'S7_' 'S8_']


In [18]:
dils = []
for dilution in dilutions: 
    files = [x for x in os.listdir(patches_dir) if f"{dilution}" in x]
    dils.append({"series": dilution, "files": files})
    print(dilution, len(files))

S11_ 3607
S12_ 1975
S19_ 19545
S1b_ 2551
S20_ 2087
S29_ 28495
S2_ 2679
S30_ 3938
S31_ 4039
S32_ 2430
S33_ 2536
S34_ 3314
S3_ 3007
S4_ 5592
S6_ 4846
S7_ 3156
S8_ 2814


In [19]:
original_dilutions = [{
    "S1b_": 95, "S2_": 75, "S3_": 50,
    "S4_": 100, "S6_": 0, "S7_": 90,
    "S8_": 25, "S11_": 99, "S12_": 0,
    "S19_": 100, "S20_": 99, "S29_": 0,
    "S30_": 95, "S31_": 50, "S32_": 90,
    "S33_": 25, "S34_": 75,
}]

In [20]:
import importlib
importlib.reload(dataset)
importlib.reload(T)


dilutions = natsorted(dilutions)
results = {}
for dil in dilutions:
    files = [os.path.join(patches_dir, x) for x in os.listdir(patches_dir) if f"{dil}" in x]
    dilution = dataset.PatchDataset(files[:3000], eval=True)
    dilution_loader = DataLoader(dilution, 512, num_workers=4)
    ims, labels, percentage = T.predict_dilution(model, dilution_loader)
    
    # fig, axs = plt.subplots(5, 5, figsize=(10,10))
    # for ax, im, l in zip(axs.ravel(), ims[:25], labels[:25]):
    #     ax.imshow(im)
    #     ax.set_title(l)
    #     ax.set_xticks([])
    #     ax.set_yticks([])
    # plt.show()
    results[dil] = percentage

    print(f"{dil} [{len(dilution)} Patches] with predicted dilution of: {percentage}")
    

Using cuda for calculations!
S1b_ [2551 Patches] with predicted dilution of: 72.56
Using cuda for calculations!
S2_ [2679 Patches] with predicted dilution of: 69.24
Using cuda for calculations!
S3_ [3000 Patches] with predicted dilution of: 42.03
Using cuda for calculations!
S4_ [3000 Patches] with predicted dilution of: 85.3
Using cuda for calculations!
S6_ [3000 Patches] with predicted dilution of: 0.2
Using cuda for calculations!
S7_ [3000 Patches] with predicted dilution of: 78.03
Using cuda for calculations!
S8_ [2814 Patches] with predicted dilution of: 19.69
Using cuda for calculations!
S11_ [3000 Patches] with predicted dilution of: 82.53
Using cuda for calculations!
S12_ [1975 Patches] with predicted dilution of: 0.2
Using cuda for calculations!
S19_ [3000 Patches] with predicted dilution of: 99.37
Using cuda for calculations!
S20_ [2087 Patches] with predicted dilution of: 95.16
Using cuda for calculations!
S29_ [3000 Patches] with predicted dilution of: 0.3
Using cuda for ca

In [21]:
df_target = pd.DataFrame(original_dilutions)
df_prediction = pd.DataFrame([results])

merged = pd.concat((df_target, df_prediction), axis=0)

merged.head()

,S1b_,S2_,S3_,S4_,S6_,S7_,S8_,S11_,S12_,S19_,S20_,S29_,S30_,S31_,S32_,S33_,S34_
0,95.00,75.00,50.00,100.0,0.0,90.00,25.00,99.00,0.0,100.00,99.00,0.0,95.00,50.00,90.0,25.00,75.00
0,72.56,69.24,42.03,85.3,0.2,78.03,19.69,82.53,0.2,99.37,95.16,0.3,94.67,51.23,91.6,7.81,77.23


In [22]:
l = {}
for i,j in merged.items():
   l[i] = j.iloc[1]-j.iloc[0]
   
merged = pd.concat((merged, pd.DataFrame([l])), axis=0)

In [23]:
merged.index = ["original", "prediction", "diff"]
merged.head()

,S1b_,S2_,S3_,S4_,S6_,S7_,S8_,S11_,S12_,S19_,S20_,S29_,S30_,S31_,S32_,S33_,S34_
original,95.00,75.00,50.00,100.0,0.0,90.00,25.00,99.00,0.0,100.00,99.00,0.0,95.00,50.00,90.0,25.00,75.00
prediction,72.56,69.24,42.03,85.3,0.2,78.03,19.69,82.53,0.2,99.37,95.16,0.3,94.67,51.23,91.6,7.81,77.23
diff,-22.44,-5.76,-7.97,-14.7,0.2,-11.97,-5.31,-16.47,0.2,-0.63,-3.84,0.3,-0.33,1.23,1.6,-17.19,2.23
